# Task 1 – K-NN classification

## Task 1.1

In [1]:
import numpy as np
import scipy.io as sio                       # for loading matlab files
from collections import Counter
import operator

In [2]:
path = "../data.mat"
data = sio.loadmat(path) # load data

(Xtrn255, Ctrn) = data['dataset']['train'][0][0][0][0] # maps to training set & labels
#                                                       (strange to have to go so deep)
(Xtst255, Ctst) = data['dataset']['test'][0][0][0][0]  # maps to test set & labels

# normalise values as directed:
Xtrn = np.array(np.divide(Xtrn255, 255.0), dtype=np.float_)
Ctrn = np.array(Ctrn, dtype=[('class', np.int_)])

Xtst = np.array(np.divide(Xtst255, 255.0), dtype=np.float_)
Ctst = np.array(Ctst, dtype=[('class', np.int_)])

In [3]:
def MySqDist(Xtrn, Xtst):
#     print(x.shape, y.shape)
#     d = x - y                   # find difference in vector components
#     return np.sum(d*d, axis=1)  # sum over squares of each row
# optimise by expressing as matrix operarions
#     (Xtrn-Xtst)^2 = Xtrn^2 + Xtst^2 - 2(Xtrn * Xtst)
    
    Xtrn_sq = np.sum(Xtrn ** 2, axis=1)[:, np.newaxis]      # lengths of all training vectors
    
    Xtst_sq = np.sum(Xtst ** 2, axis=1)                     # lengths of all test vectors
    
    Xtrn_dot_Xtst = np.dot(Xtrn, Xtst.T)                    # matrix product of sums of products
    
    return (Xtrn_sq + Xtst_sq - 2 * Xtrn_dot_Xtst).astype([('sqdist', np.float_)])

In [4]:
a = np.array([[1,2,3],
              [4,5,6]])
b = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
print(a.shape, b.shape)
# print(a)
# print(b)
%time print MySqDist(b,a)
from sklearn.metrics.pairwise import euclidean_distances
%time euclidean_distances(b, a)

((2, 3), (3, 3))
[[(  0.,) ( 27.,)]
 [( 27.,) (  0.,)]
 [(108.,) ( 27.,)]]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 725 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 337 µs


array([[ 0.        ,  5.19615242],
       [ 5.19615242,  0.        ],
       [10.39230485,  5.19615242]])

In [5]:
%time cached_cpreds = MySqDist(Xtrn, Xtst)

CPU times: user 37.6 s, sys: 10.6 s, total: 48.2 s
Wall time: 18 s


In [22]:
def my_knn_classify(Xtrn, Ctrn, Xtst, Ks):
    # Input:
    #   Xtrn : M-by-D ndarray of training data (dtype=np.float_)
    #   Ctrn : M-by-1 ndarray of labels for Xtrn (dtype=np.int_)
    #   Xtst : N-by-D ndarray of test data (dtype=np.float_)
    #   Ks   : List of the numbers of nearest neighbours in Xtrn
    # Output:
    #  Cpreds : N-by-L ndarray of predicted labels for Xtst (dtype=np.int_)
    
    Cpreds = []
    
    cl_type = [('sqdist', float), ('class', int)]           # specify end type
    
    dists = cached_cpreds
    
#     print dists
#     dists = MySqDist(Xtrn, Xtst)
    d = dists[:5000]
    print d.shape
#     print d
    
    c = Ctrn[:5000]
    cr = c.repeat(d.shape[1], axis=1)
    print c.shape
#     print c
#     print cr
    
    classed = np.column_stack((d, c))
    
    print classed
    
    cl_sort = np.argsort(classed, order=('sqdist','class'))
    
    return cl_sort
    
    print Ctrn.T.repeat(2, axis=0)[:2].shape
    print Ctrn.T.repeat(2, axis=0)[:2]
    

#     print np.dstack((dists[:2], Ctrn.T.repeat(2, axis=0)[:2]))
    
    cl_type = [('sqdist', float), ('class', int)]           # specify end type
#     classed = np.array(zip(dists, Ctrn), dtype=cl_type)     # zip distances with neighbour classes
    
#     print classed
#         sorted_c = np.sort(classed, order='sqdist')['class']
        
#         k_row = []
        
    for k in Ks:
        # add result to output var
        k_row.append(sorted(Counter(sorted_c[:k]).items())[0][0])
#             k_row.append(Counter(sorted_c[:k]))

#         Cpreds.append(k_row)
#     Cpreds.append(dists)

    return dists
        
    return Cpreds

In [23]:
# print(Xtrn[:3].shape, Xtst[0].shape)
# print(Xtrn[:3] - Xtst[0])
# %time
# m = np.array(my_knn_classify(Xtrn[:100], Ctrn[:100], Xtst[:1], 1))
# print(m.shape)
%time knn_c = my_knn_classify(Xtrn, Ctrn, Xtst, [1]) # [0][1][0]).reshape(20,5)
knn_c

(5000, 7800)
(5000, 1)


TypeError: invalid type promotion

In [ ]:
# %time knn_c = my_knn_classify(Xtrn, Ctrn, Xtst, [1,3,5,10,20]) # [0][1][0]).reshape(20,5)
# knn_c

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
%time np.square(euclidean_distances(Xtrn, Xtst))

In [24]:
# "Minimal code for rendering a numpy array as an image in a Jupyter
#  notebook in memory. Borrowed from the Deep Dream notebook."

# https://gist.github.com/kylemcdonald/2f1b9a255993bf9b2629

# used to display characters as images

import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
import string

for (i, sample) in enumerate(Xtst255[:10]):
    showarray(np.reshape(sample, (28, 28)).T)
    

    


# for pred in 
    
# np.reshape(Xtrn[0], (28, 28))

In [ ]:
a = np.arange(4).reshape((2,2))
b = a[::-1]
print a,b
c = a - b
print c
np.vdot(c, c)